## 09. 람다 라이브러리

* ramda패키지는 compose나 pipe를 사용하는 함수 조합을 쉽게할 수 있도록 설계된 라이브러리
* ramda의 특징

>1. 타입스크립트 언어와 100% 호환
>1. compose와 pipe함수 제공
>1. 자동커리(suto curry)기능 제공
>1. 포인트가 없는 고차 도움함수 제공
>1. 조합논리(combinatory theory)함수 일부 제공
>1. 하스켈 렌즈(lens) 라이브러리 기능 일부 제공
>1. 자바스크립트 표준 모나드 규격(fantasyland-spec)과 호환

### 09. 1람다 라이브러리 소개

#### ramda 패키지 구성
* ramda패키지는 많은 도움함수(utility function)를 제공한다.
* 참고사이드 
  * https://ramdajs.com/docs
  * https://devdocs.io/ramda

#### 실습프로젝트 구성

```bash
mkdir 09.ramda
cd 09.ramda
npm --init -y
npm i -D typescript ts-node @types/node

npm i -S ramda
npm i -D @types/ramda

npm i -S chance 
npm i -D @types/chanse

tsc --init

mkdir src/0902

```

##### tsconfig.json
* noImplicitAny가 false인 이유
  - 람다라이브러리는 자바스크립트를 대상으로 설계
  - 따라서, 타입스크립트는 any타입을 자바스크립트로 해석해야 하기 때문에 속성값으 false로 지정

```json
{
  "compilerOptions": {
    "module": "commonjs",
    "esModuleInterop": true,
    "target": "es2015",      
    "moduleResolution": "node",      
    "outDir": "dist",
    "baseUrl": ".",
    "sourceMap": true,
    "downlevelIteration": true,
    "noImplicitAny": false,
    "paths": { "*": ["node_modules/*"] }
  },
  "include": ["src/**/*"]
}
```

#### ramda 패키지 불러오기
* ramda패키지를 R이라는 심벌로 사용
* `R.함수명`형식으로 람다가 제공하는 함수를 사용
* `import * as R from 'ramda'`

### 09.2 람다 기본 사용법

#### R.range함수
* R.range(min, max) : min~max-1까지의 배열을 생성

```ts
import * as R from 'ramda'
console.log(R.range(1, 9+1)) // [1,2,3,4,5,6,7,8,9]
```

#### R.tab 디버깅용함수
* 2차고차함수 형태로 현잭 값을 파악할 수 있도록 한다.
* 설계대로 조함한 함수를 디버싱할 때 유용
  - `R.tab(콜백함후)(배열)`
  
##### src/0902/range-tab.ts  

```ts
import * as R from 'ramda'

const numbers: number[] = R.range(1, 9 + 1) 
R.tap(n => console.log(n))(numbers) // [1, 2, 3, 4, 5, 6, 7, 8, 9]
```

#### R.pipe 함수
* ramda에서는 compose와 pipe함수를 `R.compose, R.pipe형태로 제공`
  
##### src/0902/range-tab-pipe.ts  

```ts
import * as R from 'ramda'

const array: number[] = R.range(1, 10)
R.pipe(
  R.tap(n => console.log(n)) // [ 1, 2, 3, 4, 5, 6, 7, 8, 9  ]
)(array)
```

#### 포인트가 없는 함수
* ramda라이브러리는 대부분 2차고차함수형태로 구현되어 있다.
* 2차고차함수는 포인트가 없는 함수형태로 사용할 수 있다.

##### src/0902/dump.ts  

```ts
import * as R from 'ramda'

export const dump = R.pipe(
  R.tap(n => console.log(n))
)
```
##### src/0902/dump-test.ts  

```ts
import * as R from 'ramda'
import { dump } from './dump'

dump(R.range(1, 10)) // [ 1, 2, 3, 4, 5, 6, 7, 8, 9  ]

```
###### 포인트가 없는 함수를 화살표함수로 만들면 에러 발생
* ramda는 타입스크립트를 고려해 만든 라이브러리가 아니다.
* 포인트가 없는 함수의 모습이 낯설다고해서 일반 화살표함수로 만들면 에러 발생
* 이 오류를 해결하는 방법은 `타입단언 type assertion을 사용`하는 것.
* 결론은 포`인트가 없는 함수를 만드는 것이 불필요한 타입스크립트 오류를 만나지 않는 방안`이 된다.

```ts
// 1. 오류발생
// error TS2740: Type '{}' is missing the following properties from type 'T[]'
import * as R from 'ramda'

export const dump = <T>(array: T[]): T[] => R.pipe(
  R.tap(n => console.log(n))
)(array);

// 2. 타입단언사용
// as T[]처럼 타입단언을 사용해 R.pipe(...)(array)가 반환하는 타입을 any가 아니라 T[]로 변경
import * as R from 'ramda'

export const dump = <T>(array: T[]): T[] => R.pipe(
  R.tap(n => console.log(n))
)(array) as T[];
```
#### 자동 커리 이해하기
* 람다라이브러리 핫수는 일반함수처럼 R.add(1, 2) 매개변수가 2개인 일반함수처럼 사용할 수 있고
* R.add(1)(2)처럼 2차고차함수로 사용할 수 있다.
* 람다라이브러리에서는 이를 자동커리라고 한다.

```ts
import * as R from 'ramda'

console.log(
  R.add(1, 2), //3
  R.add(1)(2)  //3
)
```

#### R.curryN 함수
* `람다라이브러리의 함수들은 자동커리방식으로 동작할 수 있도록 매개변수의 개수가 모두 정해져 있다.`
* 따라서, 가변인수형태로 구현된 함수는 없다.

##### src/0902/sum.ts
* 가변인수로 구현된 함수(sum)

```ts
export const sum = (...numbers: number[]): number => 
  numbers.reduce((result:number, sum: number) => result + sum, 0)
```
##### src/0902/curriedSum.ts
* sum과 같은 함수를 `N차고차함수로 만들고 싶다면 R.curryN()함수를 사용`
  - `R.curryN(N, 함수)`

```ts
import * as R from 'ramda'
import {sum} from './sum'

export const curriedSum = R.curryN(4, sum)
```
##### src/0902/curriedSum-test.ts
* 마지막행을 제외한 것은 모두 부분함수이다. 따라서 출력은 [Function (anonymous)]형태로 출력

```ts
import { curriedSum } from './curriedSum'
console.log(
  curriedSum(), // [Function]
  curriedSum(1), // [Function]
  curriedSum(1)(2), // [Function]
  curriedSum(1)(2)(3), // [Function]
  curriedSum(1)(2)(3)(4) // 10
)
```

#### 순수함수
* 람다라이브러리는 순수함수를 고려해 설계되었다.
* 따라서, 람다함수들은 항상 입력변수의 상태를 변화시키지않고 새로운 값을 리턴한다.

```ts
import * as R from 'ramda'

const originalArray: number[] = [1, 2, 3]
const resultArray = R.pipe(
  R.map(R.add(1))
)(originalArray)

console.log(originalArray, resultArray) // [ 1, 2, 3 ] [ 2, 3, 4 ]
```

### 09.3 배열에 담긴 수 다루기

#### 선언형 프로그래밍

##### src/0904.inc.ts
* R.pipe함수안에서는 console문을 직접 사용할 수 없어서 R.tap함수를 사용

```ts
import * as R from 'ramda'

const numbers: number[] = R.range(1, 9 + 1)

const incNumbers = R.pipe(
  R.tap(a => console.log('before inc:', a)), // before inc: [ 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
  R.map(R.inc),
  R.tap(a => console.log('after inc:', a)), // after inc: [ 2, 3, 4,  5, 6, 7, 8, 9, 10  ]
)

const newNumbers = incNumbers(numbers)
//console.log(newNumbers) // [ 2, 3, 4,  5, 6, 7, 8, 9, 10 ]
```

#### 사칙연산함수
* R.add, R.subtract, R.multiply, R.divede 함수를 제공

```ts
// 전달받은 수에 1을 더하는 함수
// 1. 포인트가 있는 함수형태
const inc = (b: number):number => R.add(1)(b)

// 2. 포인트가 없는 함수
const inc = R.add(1)

// 3. R.map함수에 포인트가 없는 형태로 사용
// R.map(콜백함수)의 콜백함수를 익명함수로 구현한 것
R.map(
  (n:number) => inc(n)
)

// 4. 콜백함수를 간결하게 작성
R.map(inc)

// inc는 R.add(1)이기 때문에 아래처럼 작성가능
R.map(R.add(1)     
```

##### src/0903/add.ts
* 상기내용을 바탕으로 작성한 예

```ts
import * as R from 'ramda'

const incNumbers = R.pipe(
  R.map(R.add(1)),
  R.tap(a => console.log('after add(1):', a)), // after add(1): [ 2, 3, 4, 5, 6, 7, 8, 9, 10  ]
)
const newNumbers = incNumbers(R.range(1, 9 + 1))
```
#### R.addIndex함수
* Array.map은 2번쨰 매개변수로 index를 제공하지만 R.map은 index를 제공하지 않는다.
* R.map을 Array.map처럼 작동하려면 R.addIndex함수를 이용해 새로운 함수를 만들어야 한다.

```ts
const indexMap = R.addIndex(R.map)
indexMap((value: number, index: number) => R.add(number)(index))

// 콜백구현이 이전과 동일할 경우 간결하게 작성이 가능하다.
indexMap(R.add)
```

##### src/0903/addIndex.ts
* 이전 내용을 바탕으로 구현한 예

```ts
import * as R from 'ramda'

const addIndex = R.pipe(
  R.addIndex(R.map)(R.add),
  //R.addIndex(R.map)((value: number, index: number) => R.add(value)(index)),
  R.tap((a) => console.log(a)) // [ 1, 3, 5, 7, 9, 11, 13, 15, 17 ]
)
// @ts-ignore
const newNumbers = addIndex(R.range(1, 9 + 1))
```
#### R.flip함수
* R.filp함수는 매개변수의 순서를 바꿔준다.
* R.subtract(a)(b)는 a-b, R.flip(R.subtrat)은 b-a로 순서를 변경

##### src/0903/subtractFrom10.ts

```ts
import * as R from 'ramda'

const subtract = a => b => a - b

const subtractFrom10 = subtract(10)

const newArray = R.pipe(
  R.map(subtractFrom10), // 10 - value
  R.tap(a => console.log(a)) // [ 9, 8, 7, 6, 5, 4, 3, 2, 1 ]
)(R.range(1, 9+1))
```
##### src/0903/subtract.ts

```ts
import * as R from 'ramda'

const reverseSubtract = R.flip(R.subtract)

const newArray = R.pipe(
  R.map(reverseSubtract(10)), // value - 10
  R.tap(a => console.log(a)) // [ -9, -8, -7, -6, -5, -4, -3, -2, -1]
)(R.range(1, 9+1))
```
####  사칙연산함수들의 조합

##### src/0903/f.ts
* `f(x) = ax^2 + bx + c`공식을 타입스크립트로 표현

```ts
// 1. 타입스크립트 형식
type NumberToNumberFunc = (number) => number
export const f= (a: number, b: number, c: number): NumberToNumberFunc =>
                (x: number): number => a * x ** 2 + b * x + c

// 2. 람다함수 사용
import * as R from 'ramda'
    
const exp = (N:number) => (x:number):number => x ** N
cosnt square = exp(2)

type NumberToNumberFunc = (number) => number
export const f= (a: number, b: number, c: number): NumberToNumberFunc =>
    (x: number):number => R.add(a * square(x) + b * x + c)

// 3. R.multiply 적용
export const f= (a: number, b: number, c: number): NumberToNumberFunc =>
    (x: number):number => R.add(R.multiply(a)(square(x)) + R.multiply(b)(x) + c)
```

##### src/0903/f-using-ramda.ts
* 이전 코드를 R.add()함수로 대체한 R.pipe를 사용하지 않는 형태의 함수 조합의 예

```ts
import * as R from 'ramda'

export const exp = (N: number) => (x: number): number => x ** N
export const square = exp(2)

type NumberToNumberFunc = (number) => number
export const f= (a: number, b: number, c: number): NumberToNumberFunc =>
         (x: number): number => R.add( 
           R.add(
             R.multiply(a)(square(x))
           )( R.multiply(b)(x) ),  
           c
          )
```

#### 2차 방정식의 해 구현

##### src/0903/quadratic.ts
* 이전 2차함수 f에 a=1, b=2, c=1을 대입해 만든 1차함수

```ts
import {f, exp, square} from './f-using-ramda'

export const quadratic = f(1, 2, 1)
export {exp, square}
```

##### src/0903/quadratic-test.ts
* 1~10까지의 결과

```ts
import * as R from 'ramda'
import {quadratic} from './quadratic'

const input: number[] = R.range(1, 10 + 1)
const quadraticResult = R.pipe(
  R.map(quadratic),
  R.tap(a => console.log(a)) // [ 4, 9, 16, 25, 36, 49, 64, 81, 100, 121 ]
)(input)
```

##### src/0903/square-test.ts
* (x+1)^2수식에 1~10까지 대입한 예제

```ts
import * as R from 'ramda'
import {square} from './quadratic'

const input: number[] = R.range(1, 10 + 1)
const squareAferInc = R.pipe(
  R.inc, // (x + 1)
  square // (x + 1) ** 2
)
const squareResult = R.pipe(
  R.map(squareAferInc),
  R.tap(a => console.log(a)) // [ 4, 9, 16, 25, 36, 49, 64, 81, 100, 121 ]
)(input)
```

### 09.4 서술자(predicate)와 조건연산

* Array.filter함수에서 사용되는 콜백함수는 boolean타입의 값을 반환해야 하는데
* 함수형프로그래밍에서 boolean타입 값을 반환해 어떤 조건을 만족하는지를 판단하느 함수를 서술자라고 한다

#### 수의 크기를 판단하는 서술자
* 아래의 함수들은 주로 R.filter함수와 결합해 포인트가 없는 함수형태로 사용한다.

```ts
R.lt(a)(b): boolean   // a < b이면 true
R.lte(a)(b): boolean   // a <= b이면 true
R.gt(a)(b): boolean   // a > b이면 true
R.gte(a)(b): boolean   // a >= b이면 true
```
##### lte / gt / lte-gt

```ts
// src/0904.lte.ts
import * as R from 'ramda'

R.pipe(
  R.filter(R.lte(3)),
  R.tap(n => console.log(n)) // [ 3, 4, 5,  6, 7, 8, 9, 10 ]
)(R.range(1, 10 + 1))

// R.lte(3)은 직관적으로 '3 <= x'의 의미로 느껴지지 않는다.
// 그래서 R.lte(3)대신 R.flip(R.gte)처럼 사용하기도 한다.
import * as R from 'ramda'

R.pipe(
  R.filter(R.lte(3)),
  R.filter(R.gt(6 + 1)),
  R.tap(n => console.log(n)) //  [ 3, 4, 5, 6 ]
)(R.range(1, 10 + 1))

// gt.ts : 7보다 작은수만 선택
import * as R from 'ramda'

R.pipe(
  R.filter(R.gt(6 + 1)),
  R.tap(n => console.log(n)) // [ 1, 2, 3, 4, 5, 6 ]
)(R.range(1, 10 + 1))

// lte-gt.ts : 3 <= x < 7범위의 수만 선택
import * as R from 'ramda'

R.pipe(
  R.filter(R.lte(3)),
  R.filter(R.gt(6 + 1)),
  R.tap(n => console.log(n)) //  [ 3, 4, 5, 6 ]
)(R.range(1, 10 + 1))
```
#### R.allPass, R.anyPass 로직 함수
* R.lt, R.gt 처럼 boolean을 리턴하는 함수들은 R.allPass, R.anyPass함수와 결합할 수 있다.
  - R.allPass(서술자배열) : 배열의 조건을 모두 만족하면 true
  - R.anyPass(서술자배열) : 배열의 조건을 하나라도 만족하면 true

##### src/094/selectRange.ts
* min < x max

```ts
import * as R from 'ramda'

type NumberToBooleanFunc = (n: number) => boolean
export const selectRange = (min: number, max: number): NumberToBooleanFunc => 
  R.allPass([
    R.lte(min),
    R.gt(max)
  ]) 
```
##### src/094/selectRange-test.ts

```ts
import * as R from 'ramda'
import {selectRange} from './selectRange'

R.pipe(
  R.filter(selectRange(3, 6 + 1)),
  R.tap(n => console.log(n)) // [ 3, 4, 5, 6 ]
)(R.range(1, 10 + 1))
```

#### R.not 함수
* 입력값이 true면 false, false면 true를 리턴

##### src/094/notRange.ts

```ts
import * as R from 'ramda'
import {selectRange} from './selectRange'
export const notRange = (min:number, max:number) => R.pipe(selectRange(min, max), R.not)
```

##### src/094/notRange-test.ts

```ts
import * as R from 'ramda'
import {notRange} from './notRange'

R.pipe(
  R.filter(notRange(3, 6 + 1)),
  R.tap(n => console.log(n)) // [ 1, 2, 7, 8, 9, 10 ]
)(R.range(1, 10 + 1))
```

#### R.ifElse 함수
* 3개의 매개변수를 포함
  - 1st는 true/false를 리턴하는 서술자
  - 2nd는 선택자가 true일 때 실행할 함수
  - 3rd는 선택자가 fasle일 때 실행할 함수

##### src/094/ifElse.ts

```ts
import * as R from 'ramda'

const input: number[] = R.range(1, 10 + 1), halfValue = input[input.length / 2] // 6

const subtractOrAdd = R.pipe(
  R.map( R.ifElse(
    R.lte(halfValue), //x => half <= x,
    R.inc,
    R.dec,
  ) ),
  R.tap(a => console.log(a)) // [ 0, 1, 2, 3, 4, 7, 8, 9, 10, 11 ]
)
const result = subtractOrAdd(input)
```

### 09.5 문자열 다루기

#### R.trim : 문자열 앞뒤의 백색문자 자르기

```ts
import * as R from 'ramda'

console.log(
  R.trim(" \t hello \n") // hello
)
```

#### R.toLower, R.toUpper : 대소문자 전환

```ts
import * as R from 'ramda'

console.log(
  R.toUpper("Hello"), // HELLO
  R.toLower("HELLO")  // hello
)
```

#### R.split, R.join : 구분자를 사용해 문자열->문자열배열, 문자열배열->문자열로 반환
* `문자열배열 = R.split(구분자)(문자열)`
* `문자열 = R.join(구분자)(문자열배열)`

```ts
import * as R from 'ramda'

const words: string[] = R.split(' ')(`Hello world!, I'm Peter.`)
console.log(words) // [ 'Hello', 'world!,', "I'm", 'Peter.' ]
```

#### toCamelCase함수 만들기


```ts
import * as R from 'ramda'

type StringToStringFunc = (string) => string
export const toCamelCase = (delim: string): StringToStringFunc => {
  const makeFirstToCapital = (word: string) => {
    const characters = word.split('')
    return characters.map((c, index) => (index == 0 ? c.toUpperCase() : c)).join('')
  }

  const indexedMap = R.addIndex(R.map)

  return R.pipe(
    R.trim, // 문자열의 좌우 공백 문자를 제거합니다
    R.split(delim), // 문자열을 delim 문자를 근거로 배열로 전환합니다
    R.map(R.toLower), // 배열에 든 모든 문자열을 소문자열로 바꿉니다
    indexedMap(
      (value: string, index: number) => (index > 0 ? makeFirstToCapital(value) : value) // 배열에 담긴 두 번째 문자열부터 첫 문자만 대문자로 바꿉니다
    ),
    // @ts-ignore
    R.join('') // 배열을 다시 문자열로 변환합니다.
  ) as StringToStringFunc
}

// 테스트
import {toCamelCase} from './toCamelCase'

console.log(
  toCamelCase(" ")("Hello world"), // helloWorld 
  toCamelCase("_")("Hello_Albert_Einstein") // helloAlbertEinstein
)
```

### 09.6 chance패키지로 객체 만들기
* chance패키지는 `dummy데이터를 만들어 주는 라이브러리`

#### ICoodinates 타입객체 만들기

```bash
mkdir src/0906/model/coordinates
touch src/0906/model/coordinates/ICoodinates.ts
touch src/0906/model/coordinates/makeICoodinates.ts
touch src/0906/model/coordinates/makeRandomICoodinates.ts
touch src/0906/model/coordinates/index.ts
```
##### 위치기반서비스

```ts
// 1. ICoordinates.ts
export type ICoordinates = {
  latitude: number, 
  longitude: number
}

// 2. makeICoodinates.ts
import {ICoordinates} from  './ICoordinates'

export const makeICoordinates = (latitude: number, longitude: number): ICoordinates =>
  ({latitude, longitude})

// 3. makeRandomICoodinates.ts
import {ICoordinates} from './ICoordinates'
import {makeICoordinates} from './makeICoordinates'
import Chance from 'chance'
const c = new Chance

export const makeRandomICoordinates = (): ICoordinates => 
  makeICoordinates(c.latitude(), c.longitude())

// 4. index.ts
import {ICoordinates} from './ICoordinates'
import {makeICoordinates} from './makeICoordinates'
import {makeRandomICoordinates} from './makeRandomICoordinates'

// ICoordinates와 makeICoordinates, makeRandomICoordinates를 export 합니다.
export {ICoordinates, makeICoordinates, makeRandomICoordinates} 

// 5. 

```

##### src/0906/model/coordinates-test.ts

```ts
import { ICoordinates, makeRandomICoordinates } from './model/coordinates'

const coordinates: ICoordinates = makeRandomICoordinates()
console.log(coordinates) // { latitude: -33.79858, longitude: -96.83529 }
```

##### import문에서 index.ts파일은 생략가능
* ./model/coordinates 처럼 폴더만 지정했을 경우 기본으로 index.ts파일을 import한다

#### ILocation 타입객체 만들기

```bash
mkdir src/0906/model/location
touch src/0906/model/location/ILocation.ts
touch src/0906/model/location/makeILocation.ts
touch src/0906/model/location/makeRandomILocation.ts
touch src/0906/model/location/index.ts
```

```ts
// 1. ILocation.ts
import {ICoordinates} from '../coordinates'

export type ILocation = {
  country: string
  city?: string,
  address?: string,
  coordinates?: ICoordinates
}

// 2. makeILocation.ts
import {ILocation} from './ILocation'
import {ICoordinates, makeICoordinates} from '../coordinates'

export const makeILocation = (
  country: string,
  city: string,
  address: string,
  coordinates: ICoordinates
): ILocation => ({ country, city, address, coordinates })

// 3. makeRandomILocation.ts
import {ILocation} from './ILocation'
import {makeILocation} from './makeILocation'
import {makeRandomICoordinates} from '../coordinates'
import Chance from 'chance'
const c = new Chance

export const makeRandomILocation = (): ILocation => 
  makeILocation(c.country(), c.city(), c.address(), makeRandomICoordinates())

// 4. index.ts
import {ILocation} from './ILocation'
import {makeILocation} from './makeILocation'
import {makeRandomILocation} from './makeRandomILocation'

export {ILocation, makeILocation, makeRandomILocation}
```
##### src/0906/model/location-test.ts

```ts
import {makeRandomILocation, ILocation} from './model/location'

const location: ILocation = makeRandomILocation()
console.log(location) 

/*
// 실행결과
{
  country: 'CC',
  city: 'Waslivuc',
  address: '1971 Avrad Junction',
  coordinates: { latitude: 84.78191, longitude: 95.01291 }
} */
```

#### IPerson 타입객체 만들기

```bash
mkdir src/0906/model/person
touch src/0906/model/person/IPerson.ts
touch src/0906/model/person/makeIPerson.ts
touch src/0906/model/person/makeRandomIPerson.ts
touch src/0906/model/person/index.ts
```

```ts
// 1. IPerson.ts
import {ILocation} from '../location'

export type IPerson = {
  name: string,
  age: number,
  title?: string,
  location?: ILocation
}

export {ILocation}

// 2. makeIPerson.ts
import {IPerson, ILocation} from './IPerson'

export const makeIPerson = (
  name: string,
  age: number,
  title?: string,
  location?: ILocation
) => ({ name, age, title, location })

export {IPerson, ILocation}

// 3. makeRandomIPerson.ts
import { IPerson, makeIPerson } from './makeIPerson'
import { makeRandomILocation } from '../location'
import Chance from 'chance'

const c = new Chance()

export const makeRandomIPerson = (): IPerson =>
  makeIPerson(c.name(), c.age(), c.profession(), makeRandomILocation())

// 4. index.ts
import {IPerson, makeIPerson} from './makeIPerson'
import {makeRandomIPerson} from './makeRandomIPerson'

export { IPerson, makeIPerson, makeRandomIPerson }
```

##### src/0906/model/person-test.ts

```ts
import {IPerson, makeRandomIPerson} from './model/person'

const person: IPerson = makeRandomIPerson()
console.log(person) 

/*
// 실행결과
{
  name: 'Lettie Rodriquez',
  age: 29,
  title: 'Transportation Manager',
  location: {
    country: 'CR',
    city: 'Pozmoume',
    address: '1466 Nohnu View',
    coordinates: { latitude: 36.17419, longitude: 122.31801 }
  }
}
*/
```

### 09.7 렌즈를 활용한 객체의 속성 다루기
* 0906/model를 0907/model로 복사

#### 렌즈란?
* 렌즈(lens)는 하스켈언어의 Control.lens 라이브러리 중 자바스크립트에서 동작할 수 있는 
* `getter/setter 기능만을 람다함수로 구현`한 것 
* 렌즈기능
  1. R.lens함수로 객체의 특정 속성에 대한 렌즈를 만든다.
  1. 렌즈를 R.view함수에 적용해 속성값을 얻는다.
  1. 렌즈를 R.set함수에 적용해 속성값이 바뀐 새로운 객체를 얻는다.
  1. 렌즈와 속성값을 바꾸는 함수를 R.over함수에 적용해 값이 바뀐 새로운 객체를 얻는다.
  
#### R.prop와 R.assoc함수
* 렌즈기능을 이해하려면 R.prop와 R.assoc함수를 알아야 한다.
* R.prop는 'property'의미, R.prop는 객체의 특정 속성값을 가져오는 getter함수
* R.assoc함수는 특정 속성값을 변경하는 setter함수

##### src/0907/prop.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'

const person: IPerson = makeRandomIPerson()

const name = R.pipe(
  R.prop('name'),
  R.tap(name => console.log(name)) // Ollie Schwartz(랜덤하게 생성된 이름입니다)
)(person)
```

##### src/0907/assoc.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'

const getName = R.pipe(R.prop('name'), R.tap(name => console.log(name)))

const person: IPerson = makeRandomIPerson()
const originalName = getName(person) // Jeremiah Reeves

const modifiedPerson = R.assoc('name', 'Albert Einstein')(person)
const modifiedName = getName(modifiedPerson) // Albert Einstein
```

#### R.lens함수
* 렌즈기능을 사용하려면 일단 R.lens, R.prop, R.assoc함수를 조합해 렌즈를 만들어야 한다.
  - `export const makeLens = (propName: string) => R.lens(R.prop(prpopName), R.assoc(propName))`

#### R.view, R.set, R.over 함수
* 렌즈를 만들었으면 R.view, R.set, R.over 함수에 렌즈를 적용
* getter, setter, setterUsingFunc과 같은 함수를 만들 수 있다.

##### src/0907/lens.ts

```ts
import * as R from 'ramda'

export const makeLens = (propName: string) => 
  R.lens(R.prop(propName), R.assoc(propName)) 

export const getter = (lens) => R.view(lens) 
export const setter = (lens) => <T>(newValue: T) => R.set(lens, newValue)
export const setterUsingFunc = (lens) => <T, R>(func: (T) => R) => R.over(lens, func)
```

##### src/0907/lens-test.ts

```ts
import * as R from 'ramda'
import {makeLens, getter, setter, setterUsingFunc} from './lens'
import {IPerson, makeRandomIPerson} from './model/person'

const nameLens = makeLens('name')
const getName = getter(nameLens)
const setName = setter(nameLens)
const setNameUsingFunc = setterUsingFunc(nameLens)

const person: IPerson = makeRandomIPerson()

const name = getName(person)
const newPerson = setName('Albert Einstein')(person)
const anotherPerson = setNameUsingFunc(name => `'Mr. ${name}'`)(person)
const capitalPerson = setNameUsingFunc(R.toUpper)(person)

console.log(
  name, getName(newPerson), getName(anotherPerson), getName(capitalPerson)
  // Micheal Chandler Albert Einstein 'Mr. Micheal Chandler' MICHEAL CHANDLER
)

// 실행결과
// Cecelia Moss Albert Einstein 'Mr. Cecelia Moss' CECELIA MOSS
```

#### R.lensPath함수
* IPerson객체의 위경도속성값을 알려면 person.location.coordinates.longitude와 같은 코드를 작성해야 한다.
* 람다라이브러리는 객체의 이런 중첩속성(nested property)을 경로(path)라고 한다.
* 이런 긴 경로의 속성을 렌즈로 만들려면 R.lensPath함수를 사용한다.
  - `렌즈 = R.lensPath(['location', 'coordinates', 'longitude와'])`
  
##### src/0907/lensPath-test.ts

```ts
import * as R from 'ramda'
import {getter, setter, setterUsingFunc} from './lens'
import {IPerson, makeRandomIPerson} from './model/person'

const longitudeLens = R.lensPath(['location', 'coordinates', 'longitude'])
const getLongitude = getter(longitudeLens)
const setLongitude = setter(longitudeLens)
const setLongiotudeUsingFunc = setterUsingFunc(longitudeLens)

const person: IPerson = makeRandomIPerson()

const longitude = getLongitude(person)
const newPerson = setLongitude(0.1234567)(person)
const anotherPerson = setLongiotudeUsingFunc(R.add(0.1234567))(person)


console.log(
  longitude, getLongitude(newPerson), getLongitude(anotherPerson)
  // 0.17376 0.1234567 0.2972167
)
```

### 09.8. 객체다루기
* 0906/model 복사

#### R.toPairs와 R.fromParis 함수
* R.toPairs는 객체속성을 분해해 배열로 생성. 이때 배열의 각 요소는 `[string, any] 타입의 tuple`이다.
* R.fromParissms `[키, 값]형태의 요소를 가진 배열을 다시 객체로 변환`해 준다.

##### src/0906/toParis-test.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'

const person: IPerson = makeRandomIPerson()
const pairs: [string, any][] = R.toPairs(person)
console.log('pairs', pairs)
/* 
// 실행결과
pairs [
  [ 'name', 'Jeff Bridges' ],
  [ 'age', 57 ],
  [ 'title', 'Flight Engineer' ],
  [
    'location',
    {
      country: 'LY',
      city: 'Guvmevu',
      address: '701 Veuha Plaza',
      coordinates: [Object]
    }
  ]
] */
```
##### src/0906/fromParis-test.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'

const pairs: [string, any][] = R.toPairs(makeRandomIPerson())
const person: IPerson = R.fromPairs(pairs) as IPerson
console.log('person:', person) 
/* 
// 실행결과
person: {
  name: 'Sadie George',
  age: 35,
  title: 'Tax Specialist',
  location: {
    country: 'RO',
    city: 'Giwraak',
    address: '569 Hopve Lane',
    coordinates: { latitude: -34.78422, longitude: -114.44199 }
  }
} */
```

#### R.keys와 R.values함수
* R.keys함수는 객체의 속성만 string[]로 리턴
* R.values함수는 객체의 값만 any[]로 리턴

##### src/0906/model/keys-test.ts

```ts
import * as R from 'ramda'
import {makeRandomIPerson} from './model/person'

const keys: string[] = R.keys(makeRandomIPerson())
console.log('keys:', keys) // keys: [ 'name', 'age', 'title', 'location' ]
```

##### src/ 0906/values-test.ts

```ts
import * as R from 'ramda'
import {makeRandomIPerson} from './model/person'

const values: any[] = R.values(makeRandomIPerson())
console.log('values:', values) 
/* 
// 실행결과
values: [
  'Dora Young',
  62,
  'MIS Manager',
  {
    country: 'ST',
    city: 'Wuarujov',
    address: '1937 Sijuw Point',
    coordinates: { latitude: 83.83357, longitude: 12.53624 }
  }
] */
```
#### R.zipObj함수
* R.zipObj함수는 `키배열과 값배열을 결합해 객체로 변환`해 준다.
  - `객체 = R.zipObj(키배열, 값배열)`

##### src/0906/zipObj-test.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'

const originalPerson: IPerson = makeRandomIPerson()
const keys: string [] = R.keys(originalPerson)
const values: any[] = R.values(originalPerson)
const zippedPerson: IPerson = R.zipObj(keys, values) as IPerson
console.log('originalPerson:', originalPerson, 'zippedPerson:', zippedPerson) 

/* 
// 실행결과
originalPerson: {
  name: 'Amy Burgess',
  age: 63,
  title: 'Program Manager',
  location: {
    country: 'AQ',
    city: 'Hahmegsel',
    address: '154 Neser View',
    coordinates: { latitude: -77.55442, longitude: -82.01775 }
  }
} zippedPerson: {
  name: 'Amy Burgess',
  age: 63,
  title: 'Program Manager',
  location: {
    country: 'AQ',
    city: 'Hahmegsel',
    address: '154 Neser View',
    coordinates: { latitude: -77.55442, longitude: -82.01775 }
  }
} */
```
#### R.mergeLeft와 R.mergeRight함수
* R.mergeLeft와 R.mergeRight함수는 두 개의 객체를 전달받아 두 객체의 속성들을 결합해 새로운 객체로 리턴
* 속성값이 다를 경우 왼쪽객체(mergeLeft) or 오른쪽객체(mergeRight)의 우선순위가 높다

##### src/0906/mergeLeft-test.ts

```ts
import * as R from 'ramda'

const left = {name:"Jack"}, right = {name: "Jane", age: 32 }
const person = R.mergeLeft(left, right)
console.log(person) // { name: 'Jack', age: 32 }
```

##### src/0906/mergeRight-test.ts

```ts
import * as R from 'ramda'

const left = {name:"Jack"}, right = {name: "Jane", age: 32 }
const person = R.mergeRight(left, right)
console.log(person) // { name: 'Jane', age: 32 }
```

#### R.mergeDeepLeft와 R.mergeDeepRight 함수
* R.mergeLeft와 R.mergeRight함수는 객체의 속성을 변경하지 못하지만
* R.mergeDeepLeft와 R.mergeDeepRight 함수는 객체의 속성을 변경할 수 있다.

##### src/0906/mergeDeepRight-test.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'
import {ILocation, makeRandomILocation} from './model/location'
import {ICoordinates, makeRandomICoordinates} from './model/coordinates'

const person: IPerson = makeRandomIPerson()
const location: ILocation = makeRandomILocation()
const coordinates: ICoordinates = makeRandomICoordinates()

const newLocation = R.mergeDeepRight(location, {coordinates})
const newPerson = R.mergeDeepRight(person, {location: newLocation})

console.log('person:', person)
/* 
// 실행결과
person: {
  name: 'Olive Barrett',
  age: 23,
  title: 'Superintendent',
  location: {
    country: 'CG',
    city: 'Jekinvup',
    address: '596 Jojoh Circle',
    coordinates: { latitude: 28.39547, longitude: -71.19589 }
  }
} */
console.log('newPerson:', newPerson) 

/*
// 실행결과
newPerson: {
  name: 'Olive Barrett',
  age: 23,
  title: 'Superintendent',
  location: {
    country: 'TR',
    city: 'Ubsogri',
    address: '1149 Bejo Glen',
    coordinates: { latitude: -43.47301, longitude: -7.63088 }
  }
} */
```

### 09.9 배열다루기

#### R.prepend와 R.append함수
* R.prepend와 R.append함수는 기존 배열의 앞뒤에 새 요소를 삽입한 새 배열을 리턴
* 순수함수 관점에서보면 기존의 내용을 훼손하게 되기 때문에 이 함수가 만들어 졌다.

##### src/0909/prepend-test.ts

```ts
import * as R from 'ramda'

const array: number[] = [3, 4]
const newArray = R.prepend(1)(array)
console.log(array, newArray) // [ 3, 4 ] [ 1, 3, 4 ]
```

##### src/0909/append-test.ts

```ts
import * as R from 'ramda'

const array: number[] = [3, 4]
const newArray = R.append(1)(array)
console.log(array, newArray) // [ 3, 4 ] [ 3, 4, 1 ]
```

#### R.flatten함수
* 복잡한 배열을 1차원 배렬로 변환

##### src/0909/flatten-test.ts

```ts
import * as R from 'ramda'

const array = R.range(1, 2 + 1)
               .map((x: number) => {
                 return R.range(1, 2 + 1)
                  .map((y: number) => {
                    return [x, y]
                  })
               })
console.log(array) // [ [ [ 1, 1 ], [ 1, 2 ] ], [ [ 2, 1 ], [ 2, 2 ] ] ]

const flattendArray = R.flatten(array)
console.log(flattendArray) // [ 1, 1, 1, 2, 2, 1, 2, 2 ]
```

#### R.unnest함수
* R.unnest함수는 R.flaten보다 조금 정교하게 배열을 가공
* 다음 코드는 R.unnest함수를 한번 or 두번 적용했을 때의 테스트

##### src/0909/unnest-test.ts

```ts
import * as R from 'ramda'

const array = R.range(1, 2 + 1).map((x: number) => {
  return R.range(1, 2 + 1).map((y: number) => {
    return [x, y]
  })
})
console.log(array) // [ [ [ 1, 1 ], [ 1, 2 ] ], [ [ 2, 1 ], [ 2, 2 ] ] ]

const unnestedArray = R.unnest(array)
console.log(unnestedArray) // [ [ 1, 1 ], [ 1, 2 ], [ 2, 1 ], [ 2, 2 ] ]
// @ts-ignore
const twoUnnestedArray = R.pipe(R.unnest, R.unnest)(array)
console.log(twoUnnestedArray) // [ 1, 1, 1, 2, 2, 1, 2, 2 ]
```

##### R.sort함수
* `정렬된 배열 = Rsot(콜백함수)(배열)`

##### src/0909/sort-test.ts

```ts
import * as R from 'ramda'

type voidToNumberFunc = () => number
const makeRandomNumber = (max: number): voidToNumberFunc => 
  (): number => Math.floor(Math.random() * max)

const array = R.range(1, 5 + 1).map(makeRandomNumber(100))
const sortedArray = R.sort( (a:number, b): number => a - b)(array)

console.log(array, sortedArray) // [ 65, 61, 70, 12, 52 ] [ 12, 52, 61, 65, 70 ]
```

#### R.sortBy함수
* 항상 오름차순으로만 정렬
* `정렬된 배열 = R.sortBy(객체의 속성을 얻는 함수)(배열)`

##### src/0909/sortBy-test.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'
import {displayPersons} from './displayPersons'

const persons: IPerson[] = R.range(1, 4 + 1).map(makeRandomIPerson)
const nameSortedPersons = R.sortBy(R.prop('name'))(persons)
const ageSortedPersons = R.sortBy(R.prop('age'))(persons)

displayPersons('sorted by name: ')(nameSortedPersons) 
/*
sorted by name:  [
  { name: 'Adrian Elliott', age: 55 },
  { name: 'Alta Reynolds', age: 61 },
  { name: 'Curtis Rhodes', age: 43 },
  { name: 'Jerome Ortiz', age: 57 }
] 
*/

displayPersons('sorted by age: ')(ageSortedPersons) 
/* 
sorted by age:  [
  { name: 'Curtis Rhodes', age: 43 },
  { name: 'Adrian Elliott', age: 55 },
  { name: 'Jerome Ortiz', age: 57 },
  { name: 'Alta Reynolds', age: 61 }
] 
*/
```

#### R.sortWith함수
* R.ascend, R.descend함수와 같이 사용하여 오름차순, 내림차순으로 정렬

##### src/0909/sorWith-test.ts

```ts
import * as R from 'ramda'
import {IPerson, makeRandomIPerson} from './model/person'
import {displayPersons} from './displayPersons'

const persons: IPerson[] = R.range(1, 4 + 1).map(makeRandomIPerson)
const nameSortedPersons = R.sortWith([
  R.descend(R.prop('name'))
])(persons)

displayPersons('sorted by name: ')(nameSortedPersons) 
/* 
sorted by name:  [
  { name: 'Sadie Webb', age: 23 },
  { name: 'Rosetta Davidson', age: 52 },
  { name: 'Maggie Diaz', age: 23 },
  { name: 'Anne Conner', age: 41 }
] 
*/
```

### 09.10 조합논리 이해하기

* 함수형 프로그래밍의 가장 큰 이론적 배경은 람다수학과 조합논리학, 그리고 카테고리이론이다.
* 람다수학의 모든 이론을 프로그래밍언어로 표현할 수 없으므로 제한된 범위에서 `람다수학을 구현하기 위해 조합논리학`이 생겨 났다.

#### 조합자란?
* 조합논리학은 `조합자 combinator라는 특별한 형태의 고차함수들을 결합해 새로운 조합자를 만들어 내는 것`이다.
* R.chain함수를 통해 조합자들을 결합해서 사용

##### 람다가 제공하는 조합자

|조합자이름|의미|람다함수이름|
|:---:|:----------------|:---------------|
|I|identity|R.identity|
|K|constant|R.always|
|T|thrush|R.applyTo|
|W|duplication|R.unnest|
|C|flip|R.flip|
|S|substitution|R.ap|

#### R.chain 함수 탐구
* R.chain함수는 함수를 매개변수로 받아 동작하는 함수로 매개변수가 한 개 또는 두 개일 때의 동작이 조금 다르다.
  - R.chain(콜백함수)
  - R.chain(콜백함수, 콜백함수)

##### src/0910/chain-test.ts
* R.head함수는 배열(list or string)에서 첫 번째 요소를 리턴

```ts
import * as R from 'ramda'

const array = [1, 2, 3]

R.pipe(
  R.chain(n => [n, n]),
  R.tap(n => console.log(n)) // [ 1, 1, 2, 2, 3, 3 ]
)(array)

R.pipe(
  R.chain(R.append, R.head),
  R.tap(n => console.log(n)) // [ 1, 2, 3, 1 ]
)(array)
```
##### src/0910/flatMap.ts
* R.chain함수는 `매개변수가 한 개일 때 flapMap함수처럼 동작`한다.

```ts
import * as R from 'ramda'

export const flatMap = (f) => R.pipe(
  R.map(f),
  R.flatten
)
```
##### src/0910/flatMap-test.ts

```ts
import * as R from 'ramda'
import {flatMap} from './flatMap'

const array = [1, 2, 3]
R.pipe(
  flatMap(n => [n, n]),
  R.tap(n => console.log(n))
)(array)
```

##### src/0910/chainTwoFunc.ts
* R.chain함수는 `매개변수가 두 개일 때 chainTwoFunc함수처럼 동작`한다.

```ts
import * as R from 'ramda'
export const chainTwoFunc = (firstFn, secondFn) => (x) => firstFn(secondFn(x), x)
```
##### src/0910/chainTwoFunc-test.ts

```ts
import * as R from 'ramda'
import {chainTwoFunc} from './chainTwoFunc'

const array = [1, 2, 3]
R.pipe(
  chainTwoFunc(R.append, R.head), // array => R.append(R.head(array))(array)
  R.tap(n => console.log(n)) // [ 1, 2, 3, 1 ]
)(array)
```

#### R.flip 조합자
* 2차 고차함수 매개변수의 순서를 바꾸는 역할

##### src/0910/flip-test.ts

```ts
import * as R from 'ramda'

const flip = cb => a => b => cb(b)(a)
// const flip =  cb => { return a => { return b => cb(b)(a) }}
const reverseSubtract = flip(R.subtract)

const newArray = R.pipe(
  R.map(reverseSubtract(10)), // value - 10
  R.tap(a => console.log(a)) // [ -9, -8, -7, -6, -5, -4, -3, -2, -1]
)(R.range(1, 9+1))
```

#### R.identity 조합자

##### src/0910/funnest-using-flatMap.ts
* R.identity 사용한 예

```ts
import * as R from 'ramda'
import {flatMap} from './flatMap'

const unnest = flatMap(R.identity)

const array = [ [1], [2], [3] ]
R.pipe(
  unnest,
  R.tap(n => console.log(n)) // [ 1, 2, 3 ]
)(array)
```
##### src/0910/unnest-using-flatMap.ts
* R.ifElse 사용한 예

```ts
import * as R from "ramda"

type NumtoNumFunc = (n:number) => number
const applyDiscount = (minimum: number, discount: number): NumtoNumFunc =>
  R.pipe(
    R.ifElse( 
      R.flip(R.gte)( minimum ), 
      R.flip(R.subtract)( discount ), 
      R.identity
    ),
    R.tap(amount => console.log(amount))
  )
const calcPrice = applyDiscount(5000, 500) 

const discountedPrice = calcPrice( 6000 ) // 5500
const notDiscountedPrice = calcPrice(4500) // 4500
```

#### R.always 조합자
* 2개의 고차매개변수중 첫 번째 것을 반환
* 이런 특징 때문에 R.always조합자는 constant라는 의미에서 K-조합자라고 한다.
* K는 독일어로 Konstante(상수)를 의미한다.

##### src/0910/first-second.ts

```ts
import * as R from 'ramda'

const always = a => b => a
const flip = cb => a => b => cb(b)(a)

const first = <T>(a: T) => (b: T): T => always(a)(b)
const second = <T>(a: T) => (b: T): T => flip(always)(a)(b)

console.log(
  first(1)(2), // 1
  second(1)(2) // 2
)
```

#### R.applyTo 조합자
* 값을 첫 번쨰 매개변수로 받고 이 값을 입력으로 하는 콜백함수를 2번째 매개변수로 받는다.
  - `const applyTo = value => cb => cb(value)` 형태로 동작
  
##### src/0910/applyTo-test.ts

```ts
import * as R from 'ramda'

const T = value => R.pipe(
  R.applyTo(value),
  R.tap(value => console.log(value))
)

const value100 = T(100)
const sameValue = value100(R.identity) // 100
const add1Value = value100(R.add(1)) // 101
```

#### R.ap 조합자
* 콜백함수들의 배열을 첫 번째 매개변수로, 배열을 두 번째 매개변수로 입력받은 2차 고차함수
  - `const ap = ([콜백함수]) => 배열 => [콜백함수](배열)`
* R.ap는 콜백함수가 한 개일 떄는 마치 R.map 함수처럼 동작한다.
* 두 개일 경우 마치 R.chain(n => [n, n])처럼 동작

##### src/0910/ap-test.ts


```ts
import * as R from 'ramda'

const callAndAppend = R.pipe(
  R.ap([R.multiply(2)]),
  R.tap(a => console.log(a))
)

const input = [1, 2, 3]
const result = callAndAppend(input) // [ 2, 4, 6 ]
```


##### src/0910/ap-test.ts
* 콜백함수가 한 개일 경우

```ts
import * as R from 'ramda'

const callAndAppend = R.pipe(
  R.ap([R.multiply(2)]),
  R.tap(a => console.log(a))
)

const input = [1, 2, 3]
const result = callAndAppend(input) // [ 2, 4, 6 ]
```

##### src/0910/ap-test2.ts
* 콜백함수가 두 개일 경우
* 두 콜백함수를 적용한 각각의 배열을 만든 후 연산이 종료가 되면 배열을 통합해 한 개의 배열로 리턴

```ts
import * as R from 'ramda'

const callAndAppend = R.pipe(
  R.ap([R.multiply(2), R.add(10)]),
  R.tap(a => console.log(a))
)

const input = [1, 2, 3]
const result = callAndAppend(input) // [ 2, 4, 6, 11, 12, 13 ]
```

##### src/0910/ap-test3.ts
* [1,2,3]배열을 3번 복제후 한 개의 배열로 통합하는 예제

```ts
import * as R from 'ramda'

const repeat = (N, cb) => R.range(1, N + 1).map(n => cb)

const callAndAppend = R.pipe(
  R.ap(repeat(3, R.identity)),
  R.tap(a => console.log(a))
)

const input = [1, 2, 3]
const result = callAndAppend(input) // [ 1, 2, 3, 1, 2, 3, 1, 2, 3 ]
```